In [2]:
import hopsworks
from dotenv import load_dotenv
import os

# Cargar la API Key desde el archivo .env
load_dotenv()
api_key = os.getenv("HOPSWORKS_API_KEY")

# Conectar a Hopsworks
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

# Obtener el Feature Group titanic_features_updated
titanic_fg = fs.get_feature_group(name="titanic_features_updated", version=1)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1047704
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Cargar las características desde el Feature Store
features = titanic_fg.read()

# Mostrar las primeras filas para verificar que los datos han sido cargados correctamente
features.head()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.21s) 


,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,familysize,embarked_c,embarked_q,embarked_s
0,866,1,2,"Bystrom, Mrs. (Karolina)",1,42.000000,0,0,236852,13.0000,1,False,False,True
1,726,0,3,"Oreskovic, Mr. Luka",0,20.000000,0,0,315094,8.6625,1,False,False,True
2,159,0,3,"Smiljanic, Mr. Mile",0,29.699118,0,0,315037,8.6625,1,False,False,True
3,587,0,2,"Jarvis, Mr. John Denzil",0,47.000000,0,0,237565,15.0000,1,False,False,True
4,848,0,3,"Markoff, Mr. Marin",0,35.000000,0,0,349213,7.8958,1,True,False,False


In [4]:
from sklearn.model_selection import train_test_split

# Dividir las características (X) y la variable objetivo (y)
X = features.drop(columns=['survived'])  # Eliminar la columna objetivo
y = features['survived']  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
from lazypredict.Supervised import LazyClassifier

# Crear el LazyClassifier
clf = LazyClassifier()

# Entrenar los modelos
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Mostrar los resultados de los modelos
models


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  3.80it/s]

[LightGBM] [Info] Number of positive: 277, number of negative: 435
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000796 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 923
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.389045 -> initscore=-0.451329
[LightGBM] [Info] Start training from score -0.451329
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,


In [6]:
# Ordenar los modelos por Accuracy o F1 Score (elige la métrica que prefieras)
best_model = models.sort_values(by='Accuracy', ascending=False).head(1)

# Mostrar el mejor modelo
best_model


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,


In [8]:
X_train.head()
y_train.head()


331    1
733    0
382    1
704    0
813    0
Name: survived, dtype: int64

In [18]:
print(X_train.columns)


Index(['passengerid', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'familysize', 'embarked_c', 'embarked_q',
       'embarked_s'],
      dtype='object')


In [19]:
X_train = X_train.drop(columns=['Name', 'Ticket'], errors='ignore')
X_test = X_test.drop(columns=['Name', 'Ticket'], errors='ignore')


In [21]:
import pandas as pd

# Verificar si hay columnas categóricas y usar get_dummies
X_train = pd.get_dummies(X_train, columns=['sex', 'embarked_c', 'embarked_q', 'embarked_s'])
X_test = pd.get_dummies(X_test, columns=['sex', 'embarked_c', 'embarked_q', 'embarked_s'])


In [24]:
X_train = X_train.drop(columns=['name', 'ticket'], errors='ignore')
X_test = X_test.drop(columns=['name', 'ticket'], errors='ignore')


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Entrenar un modelo de Regresión Logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Realizar predicciones
y_pred_logreg = logreg.predict(X_test)

# Evaluar el modelo
print("Accuracy del modelo de Regresión Logística:", accuracy_score(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))


Accuracy del modelo de Regresión Logística: 0.8156424581005587
              precision    recall  f1-score   support

           0       0.85      0.86      0.86       114
           1       0.75      0.74      0.74        65

    accuracy                           0.82       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.82      0.82      0.82       179



In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

# Entrenar un modelo de Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Accuracy del modelo Random Forest:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

# Entrenar un modelo de SVM
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("\nAccuracy del modelo SVM:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

# Entrenar un modelo de K-Nearest Neighbors
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("\nAccuracy del modelo KNN:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))


Accuracy del modelo Random Forest: 0.8156424581005587
              precision    recall  f1-score   support

           0       0.85      0.86      0.86       114
           1       0.75      0.74      0.74        65

    accuracy                           0.82       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.82      0.82      0.82       179


Accuracy del modelo SVM: 0.659217877094972
              precision    recall  f1-score   support

           0       0.66      0.97      0.78       114
           1       0.70      0.11      0.19        65

    accuracy                           0.66       179
   macro avg       0.68      0.54      0.49       179
weighted avg       0.67      0.66      0.57       179


Accuracy del modelo KNN: 0.6089385474860335
              precision    recall  f1-score   support

           0       0.67      0.76      0.71       114
           1       0.45      0.34      0.39        65

    accuracy                           

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definir el modelo de Random Forest
rf = RandomForestClassifier(random_state=42)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Configurar GridSearchCV con validación cruzada
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)

# Entrenar GridSearchCV
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros: ", grid_search.best_params_)

# Evaluar el mejor modelo en los datos de prueba
best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)

# Mostrar el reporte de clasificación
from sklearn.metrics import classification_report
print("Accuracy del mejor modelo de Random Forest optimizado:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Mejores hiperparámetros:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Accuracy del mejor modelo de Random Forest optimizado: 0.8212290502793296
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       114
           1       0.77      0.72      0.75        65

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.82      0.82      0.82       179



In [30]:
import joblib

# Guardar el modelo entrenado en un archivo
joblib.dump(best_rf, 'modelo_random_forest_optimizado.pkl')


['modelo_random_forest_optimizado.pkl']

In [33]:
# Cargar el modelo guardado
modelo_cargado = joblib.load('modelo_random_forest_optimizado.pkl')

# Crear un pipeline de preprocesamiento y predicción
def predecir(datos_nuevos):
    # Convertir las columnas categóricas a True/False en lugar de 1/0
    datos_nuevos['sex'] = datos_nuevos['sex'].map({0: False, 1: True})
    datos_nuevos['embarked_c'] = datos_nuevos['embarked_c'].map({0: False, 1: True})
    datos_nuevos['embarked_q'] = datos_nuevos['embarked_q'].map({0: False, 1: True})
    datos_nuevos['embarked_s'] = datos_nuevos['embarked_s'].map({0: False, 1: True})
    
    # Asegurarse de que las columnas coincidan con las del entrenamiento
    datos_nuevos = pd.get_dummies(datos_nuevos, columns=['sex', 'embarked_c', 'embarked_q', 'embarked_s'])
    
    # Rellenar las columnas faltantes con ceros para coincidir con las columnas del modelo entrenado
    columnas_entrenamiento = modelo_cargado.feature_names_in_  # Asegúrate de que esto sea compatible con tu modelo
    datos_nuevos = datos_nuevos.reindex(columns=columnas_entrenamiento, fill_value=0)
    
    # Realizar la predicción
    prediccion = modelo_cargado.predict(datos_nuevos)
    
    return prediccion

# Ejemplo de uso con datos nuevos
datos_nuevos = pd.DataFrame({
    'pclass': [1],
    'sex': [0],  # Aquí 0 para 'male', que se mapeará a False
    'age': [29],
    'sibsp': [1],
    'parch': [0],
    'fare': [50],
    'familysize': [1],
    'embarked_c': [1],  # Se mapeará a True
    'embarked_q': [0],  # Se mapeará a False
    'embarked_s': [0]   # Se mapeará a False
})

# Obtener la predicción
prediccion = predecir(datos_nuevos)
print(f"Predicción: {prediccion}")



Predicción: [1]
